# Results regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import copy

# torch
import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.utils import data
import torchvision
from torchvision import transforms
import torch.multiprocessing as mp

from pandas import DataFrame

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection

from numpy import savetxt
from numpy import loadtxt

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Cuda is available: ",torch.cuda.is_available())

## Regression data

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

cal_housing = fetch_california_housing()
#X = cal_housing.data
sigma = cal_housing.target.std()**2
feature_names = cal_housing.feature_names
n_features = len(feature_names)
num_output = 1
#y -= y.mean()
#y /= y.std()
#scaler = StandardScaler()
#X = scaler.fit_transform(X, y)

# save problem
#savetxt('data/X_regression.csv', X, delimiter=',')
#savetxt('data/y_regression.csv', y, delimiter=',')
#X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size=0.15, random_state=0)
#X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size=0.15, random_state=0)
#savetxt('data/X_test_regression.csv', X_test, delimiter=',')
#savetxt('data/y_test_regression.csv', y_test, delimiter=',')
#savetxt('data/X_val_regression.csv', X_val, delimiter=',')
#savetxt('data/y_val_regression.csv', y_val, delimiter=',')
#savetxt('data/X_train_regression.csv', X_train, delimiter=',')
#savetxt('data/y_train_regression.csv', y_train, delimiter=',')

# fetch classification problem
X = loadtxt('data/problem/X_regression.csv',delimiter=',')
y = loadtxt('data/problem/y_regression.csv',delimiter=',')
X_test = loadtxt('data/problem/X_test_regression.csv',delimiter=',')
y_test = loadtxt('data/problem/y_test_regression.csv',delimiter=',')
X_val = loadtxt('data/problem/X_val_regression.csv',delimiter=',')
y_val = loadtxt('data/problem/y_val_regression.csv',delimiter=',')
X_train = loadtxt('data/problem/X_train_regression.csv',delimiter=',')
y_train = loadtxt('data/problem/y_train_regression.csv',delimiter=',')

In [ ]:
# create the 2d vandermonde vectors (will be reshaped to 3d later when necessary)
def vandermonde_vec(dataset, n_instances, n_features, poly_order):
    u = np.zeros((n_instances, n_features*poly_order))
    
    # Get powers
    for row in range(n_instances):
        for col in range(n_features):
            u[row,col*poly_order:(col+1)*poly_order] = np.power(
                [dataset[row,col]]*poly_order, list(range(poly_order)))
    return u

## Models

### Tensor train

In [ ]:
class TTNet(nn.Module):

    def __init__(self, n_features, poly_order, num_output, rank):
        super(TTNet, self).__init__()  
        
        self.n_features = n_features
        self.poly_order = poly_order
        self.num_output = num_output
        self.rank = rank
        self.type = 'tt'

        Di = self.rank
        Dn = self.rank
        # Elements are drawn from a uniform distribution [-1/sqrt(D),1/sqrt(D)]
        bound_i = 1/np.sqrt(Di)
        bound_n = 1/np.sqrt(Dn)
        # bounds on the uniform distribution
        lb = 0.5*bound_i
        ub = 1.0*bound_i

        # input layer
        self.tt_cores = []
        for i in range(n_features):
            if i==0: 
                tn_size = (1,poly_order,self.rank)
            elif i==n_features-1:
                tn_size = (self.rank,poly_order,num_output)
            else:
                tn_size = (self.rank,poly_order,self.rank)
            
            k = 1/(np.sqrt(self.poly_order))
            g_i = Parameter(init.normal_(torch.empty(tn_size, requires_grad=True), mean=0, std=1)*k)
            self.tt_cores.append(g_i)
            
        self.tt_cores = nn.ParameterList(self.tt_cores)

    def get_n_params(self):
        pp=0
        for p in list(self.parameters()):
            nn=1
            for s in list(p.size()):
                nn = nn*s
            pp += nn
        return pp

    def forward(self, vec_input, batch_size, print_expr=False):
                
        vec = vec_input[:,:self.poly_order].reshape(batch_size,-1)
        
        # First do: G_i x_2 v_i
        mode2 = []
        for i in range(self.n_features):
            vec = vec_input[:,i*self.poly_order:(i+1)*self.poly_order].reshape(batch_size,-1)
            mode2.append(torch.einsum('abc,db -> dac', self.tt_cores[i], vec) )
            
        mode2[0] = mode2[0].reshape(batch_size,self.rank)
        mode2[-1] = mode2[-1].reshape(batch_size,self.rank,self.num_output)
        
        # Join all the results (based on equation 11 in the paper)
        result = mode2[0]
        for i in range(self.n_features-1):
            result = torch.einsum('ab,abd -> ad', result, mode2[i+1]) * 1/self.rank
        return result

### Tensor ring

In [ ]:
class TRNet(nn.Module):
    
    def __init__(self, n_features, poly_order, num_output, rank):
        super(TRNet, self).__init__()  
        
        self.n_features = n_features
        self.poly_order = poly_order
        self.num_output = num_output
        self.rank = rank
        self.outer_dim = rank # outer dimensions of the cores, G
        self.type = 'tr'
        
        # Specify the dimensions of the core tensors
        # Here, they are all of order 3 and the dimension of the first and last mode is 3
        gi_size = tuple([self.outer_dim,poly_order,self.outer_dim]) # Dimension of the core tensors except the last
        gn_size = tuple([self.outer_dim,num_output,self.outer_dim]) # Dimension of the last cores tensor
        gstack_size = tuple([self.outer_dim,poly_order,self.outer_dim,n_features]) # Dimension of the stack of cores
        
        # Elements are drawn from a uniform distribution [-1/sqrt(D),1/sqrt(D)],
        # where D is the outer dimensions of the core
        bound_i = 1/math.sqrt(self.outer_dim)
        # bounds on the uniform distribution
        lb = 0.1*bound_i
        ub = 1.0*bound_i
        
        # The cores are now combined to give one long dimension which matched the one from vandermonde
        self.Gstack = Parameter(init.uniform_(torch.empty(gstack_size, requires_grad=True),a=lb,b=ub))
        
        # The last tensor as a different size as the inner dimension is the number of classes
        self.GN = Parameter(init.uniform_(torch.empty(gn_size, requires_grad=True),a=lb,b=ub))

    def forward(self, tensor_input, batch_size, print_expr=False):
        
        # Multiplication of Vandermonde vectors
        Gv_stack= torch.einsum('abcd, edb -> aecd',self.Gstack,tensor_input)
        
        # Multiplication of the cores to get f: G_i-1 x_31 G_i
        f_stack = Gv_stack[:,:,:,0]
        
        # The multiplication by the last core and the ring multiplication is not in the for-loop
        for i in range(1,self.n_features):
            f_stack = torch.einsum('abc, cbe -> abe',f_stack,Gv_stack[:,:,:,i])

        f_stack = torch.einsum('abc, cda -> bd',f_stack, self.GN)
        return f_stack

### CPD

In [ ]:
class CPNet(nn.Module):

    def __init__(self, n_features, poly_order, num_output, rank):
        super(CPNet, self).__init__()  
        
        self.rank = rank
        self.n_features = n_features
        self.poly_order = poly_order
        self.num_output = num_output
        self.type = 'cp'
        
        # weight tensors collected in one tensor
        tn_size = tuple([n_features] + [poly_order] + [rank] + [num_output]) # size of all tensors A_i
        self.A = Parameter(init.normal_(torch.empty(tn_size, requires_grad=True), std=0.575))

    def forward(self, vec_input, batch_size, print_expr=False):
        
        m = torch.einsum('abcd,eab->aced',self.A ,vec_input)
        f = torch.prod(m,0)
        
        return torch.sum(f,0)

## Train model

In [ ]:
# train regression model
def train_regression_model(net, optimizer, criterion, num_epochs, n_features, X_train, X_val, y_train, y_val):
    
    n_train_samples = len(X_train[:,0])
    n_val_samples = len(X_val[:,0])
    
    # Training the model
    X_train = vandermonde_vec(X_train, n_train_samples, n_features, net.poly_order)
    X_val = vandermonde_vec(X_val, n_val_samples, n_features, net.poly_order)
    
    # reshape X_train, X_val to 3d vandermonde for tensor ring and cpd
    if (net.type == 'tr') or (net.type == 'cp'):
        X_train = np.reshape(X_train, (n_train_samples, n_features, net.poly_order))
        X_val = np.reshape(X_val, (n_val_samples, n_features, net.poly_order))
    
    # setting up lists for handling loss/accuracy
    train_err = -1*np.ones(num_epochs)
    valid_err = -1*np.ones(num_epochs)
    loss = 0
    losses = []
    valid_err_cur_best = 1e10
    min_val_output = np.zeros(n_val_samples)
    best_model = copy.deepcopy(net)

    for epoch in range(num_epochs):
        # Forward -> Backprob -> Update params
        ### Train
        net.train()
        loss = 0
        count = 0

        # Get data
        data = torch.Tensor(X_train).to(device)
        targets = torch.Tensor(y_train)
        count += 1

        # Transfer training data and targets to device
        targets = Variable(targets).to(device)

        # Send it through the model
        output = net(data, n_train_samples).view(n_train_samples)
        
        # compute gradients given loss
        loss = criterion(output, targets)
        optimizer.zero_grad()
        loss.backward()
        #optimizer.step()
        
        def closure():
            optimizer.zero_grad()
            output = net(data, n_train_samples).view(n_train_samples)
            loss = criterion(output, targets)
            loss.backward()
            return loss
        optimizer.step(closure)
        
        losses.append(loss)
        
        #eval
        net.eval()
        
        ### Evaluate training
        train_preds, train_targs = [], []

        #for data, labels in training_generator:
        train_data = torch.Tensor(X_train).to(device)
        train_targets = torch.Tensor(y_train)
        
        train_output = net(train_data, n_train_samples).view(n_train_samples)
        train_preds += list(train_output.data.cpu().numpy())
        train_targs += list(train_targets)
        
        ### Evaluate validation
        val_preds, val_targs = [], []

        #for data, labels in validation_generator:
        val_data = torch.Tensor(X_val).to(device)
        val_targets = torch.Tensor(y_val)

        val_output = net(val_data, n_val_samples).view(n_val_samples)
        val_preds += list(val_output.data.cpu().numpy())
        val_targs += list(val_targets)
        
        if np.isnan(train_output[0].detach().cpu().numpy()):
            return best_model, train_err, valid_err, valid_err_cur_best#, min_val_output

        train_err_cur = mean_squared_error(np.asarray(train_targs),np.asarray(train_preds))
        valid_err_cur = mean_squared_error(np.asarray(val_targs),np.asarray(val_preds))
        
        if valid_err_cur < valid_err_cur_best:
            best_model = copy.deepcopy(net)
            valid_err_cur_best = valid_err_cur
            min_val_output = val_output.data.cpu().numpy()

        train_err[epoch] = train_err_cur
        valid_err[epoch] = valid_err_cur
        print("Epoch %2i : Train Err %f" % (epoch+1, train_err_cur))
        #if epoch % (np.floor(num_epochs/50)) == 0:
            #print("Epoch %2i : Train Err %f" % (epoch+1, train_err_cur))
        #if epoch == num_epochs-1:
            #print("Last epoch %2i : Final Train Err %f" % (epoch+1, train_err_cur))
        
    return best_model, train_err, valid_err, valid_err_cur_best#, min_val_output

## Compare optimizers

In [ ]:
rank = 5
poly_order = 2
criterion = nn.MSELoss()

num_epochs = 500
runs = 5

train_errs = -1*np.ones((runs, num_epochs))

for k in range(runs):
    net = TTNet(n_features, poly_order, num_output, rank)
    optimizer = optim.Adam(net.parameters(),lr=0.1)
    #optimizer = optim.LBFGS(net.parameters(), max_iter=20, line_search_fn='strong_wolfe')
    #optimizer = optim.SGD(net.parameters(), lr=0.2)
    trained_model, train_err, valid_err = train_regression_model(net, optimizer, criterion, 
                                num_epochs, n_features, X_train, X_val, y_train, y_val)
    
    train_errs[k,0:len(train_err)] = train_err

#savetxt('data/show_same_train_err_r5_n2_Adam.csv', train_errs, delimiter=',')

## Cross-validation for poly_order and rank

In [ ]:
def crossvalidation_regression(models, K, X_train, X_val, y_train, y_val):
    S = len(models)
    
    CV = model_selection.KFold(n_splits=K, shuffle=True, random_state=0)
    
    # observations to do CV on
    X_CV = np.concatenate((X_train, X_val))
    y_CV = np.concatenate((y_train, y_val))
    
    opt_models = []
    min_errors = []
    
    valid_errors = -1*np.ones(K*S)
    num_epochs = 100
    
    k=0
    for par_index, test_index in CV.split(X_CV):
        print('Computing CV fold: {0}/{1}..'.format(k+1,K))
        
        # extract training and test set for current CV fold
        X_tr, y_tr = X_CV[par_index,:], y_CV[par_index]
        X_va, y_va = X_CV[test_index,:], y_CV[test_index]
        
        min_error = 1e15
        opt_model = None
        
        for s in range(S):
            model = models[s]
            
            poly_order = model.poly_order
            
            # define optimizer and loss criterion
            optimizer = optim.LBFGS(model.parameters())
            criterion = nn.MSELoss()
                
            # train net
            model_trained, train_err, valid_err, valid_err_cur_best = train_regression_model(
                model, optimizer, criterion, num_epochs, n_features, X_tr, X_va, y_tr, y_va)
            
            valid_errors[k*S+s] = valid_err_cur_best
            
            error = valid_err_cur_best
            if error < min_error:
                min_error = error
                opt_model = model_trained
        opt_models.append(opt_model)
        min_errors.append(min_error)
        k+=1
    return opt_models, min_errors, valid_errors

### Cross validation TT

In [ ]:
ranks = range(3,15)
poly_orders = range(1,4)
models = []
for rank in ranks:
    for poly_order in poly_orders:
        net = TTNet(n_features, poly_order, num_output, rank)
        net.to(device)
        models.append(net)

K = 3
opt_models, opt_models_min_errors, valid_errors = crossvalidation_regression(
    models, K, X_train, X_val, y_train, y_val)

#savetxt('data/cv_ranks.csv',ranks,delimiter=',')
#savetxt('data/cv_poly_orders.csv',poly_orders,delimiter=',')
#savetxt('data/cv_ttreg_valid_errors.csv',valid_errors,delimiter=',')

opt_models_poly_order = []
opt_models_rank = []
for i in range(K):
    opt_models_poly_order.append(opt_models[i].poly_order)
    opt_models_rank.append(opt_models[i].rank)
    print('optimal poly_order:', opt_models[i].poly_order)
    print('optimal rank:', opt_models[i].rank)
    print('minimal validation error of optimal model:', opt_models_min_errors[i])
    
#savetxt('data/cv_ttreg_opt_models_poly_order.csv', opt_models_poly_order,delimiter=',')
#savetxt('data/cv_ttreg_opt_models_rank.csv', opt_models_rank,delimiter=',')
#savetxt('data/cv_ttreg_opt_models_min_errors.csv', opt_models_min_errors,delimiter=',')

### Cross validation TR

In [ ]:
ranks = loadtxt('data/cv_ranks.csv',delimiter=',')#range(3,15)
poly_orders = loadtxt('data/cv_poly_orders.csv',delimiter=',')#range(1,4)
print(ranks)
print(poly_orders)
models = []
for rank in ranks:
    for poly_order in poly_orders:
        net = TRNet(int(n_features), int(poly_order), int(num_output), int(rank))
        net.to(device)
        models.append(net)

K = 3
opt_models, opt_models_min_errors, valid_errors = crossvalidation_regression(
    models, K, X_train, X_val, y_train, y_val)

#savetxt('data/cv_ranks.csv',ranks,delimiter=',')
#savetxt('data/cv_poly_orders.csv',poly_orders,delimiter=',')
#savetxt('data/cv_trreg_valid_errors.csv',valid_errors,delimiter=',')

opt_models_poly_order = []
opt_models_rank = []
for i in range(K):
    opt_models_poly_order.append(opt_models[i].poly_order)
    opt_models_rank.append(opt_models[i].rank)
    print('optimal poly_order:', opt_models[i].poly_order)
    print('optimal rank:', opt_models[i].rank)
    print('minimal validation error of optimal model:', opt_models_min_errors[i])
    
#savetxt('data/cv_trreg_opt_models_poly_order.csv', opt_models_poly_order,delimiter=',')
#savetxt('data/cv_trreg_opt_models_rank.csv', opt_models_rank,delimiter=',')
#savetxt('data/cv_trreg_opt_models_min_errors.csv', opt_models_min_errors,delimiter=',')

### Cross validation CP

In [ ]:
ranks = loadtxt('data/cv_ranks.csv',delimiter=',')#range(3,15)
poly_orders = loadtxt('data/cv_poly_orders.csv',delimiter=',')#range(1,4)
print(ranks)
print(poly_orders)
models = []
for rank in ranks:
    for poly_order in poly_orders:
        net = CPNet(int(n_features), int(poly_order), int(num_output), int(rank))
        net.to(device)
        models.append(net)

K = 3
opt_models, opt_models_min_errors, valid_errors = crossvalidation_regression(
    models, K, X_train, X_val, y_train, y_val)

#savetxt('data/cv_ranks.csv',ranks,delimiter=',')
#savetxt('data/cv_poly_orders.csv',poly_orders,delimiter=',')
#savetxt('data/cv_cpreg_valid_errors.csv',valid_errors,delimiter=',')

opt_models_poly_order = []
opt_models_rank = []
for i in range(K):
    opt_models_poly_order.append(opt_models[i].poly_order)
    opt_models_rank.append(opt_models[i].rank)
    print('optimal poly_order:', opt_models[i].poly_order)
    print('optimal rank:', opt_models[i].rank)
    print('minimal validation error of optimal model:', opt_models_min_errors[i])
    
#savetxt('data/cv_cpreg_opt_models_poly_order.csv', opt_models_poly_order,delimiter=',')
#savetxt('data/cv_cpreg_opt_models_rank.csv', opt_models_rank,delimiter=',')
#savetxt('data/cv_cpreg_opt_models_min_errors.csv', opt_models_min_errors,delimiter=',')

# Rank vs weight decay

In [ ]:
# find optimal weight decays for each rank
num_epochs = 100
poly_order = 2
ranks = np.arange(1,11) #[1,...,10]
weights = np.asarray(
    [0, 1.e-10, 1.e-9, 1.e-8, 1.e-7, 1.e-6, 1.e-5, 1.e-4, 1.e-3, 1.e-2, 1.e-1, 1.e+0])

# run model with adam optim for each weight and each rank. see which weights gives the best validation error

models = []
for rank in ranks:
    model = TTNet(n_features, poly_order, num_output, rank)
    model.to(device)
    models.append(model)

train_err_models = np.zeros((len(models)*len(weights), num_epochs))
valid_err_models = np.zeros((len(models)*len(weights), num_epochs))

counter = 0
for i in range(len(models)):
    model = models[i]
    for j in range(len(weights)):
        print("Model:",i+1,j)
        weight = weights[j]
        #optimizer = optim.LBFGS(model.parameters(), line_search_fn='strong_wolfe')
        optimizer = optim.Adam(model.parameters(), lr=0.1, weight_decay=weight)
        criterion = nn.MSELoss()
        trained_model, train_err, valid_err, valid_err_cur_best = train_regression_model(
            model, optimizer, criterion, num_epochs, n_features, X_train, X_val, y_train, y_val)
        train_err_models[counter] = train_err
        valid_err_models[counter] = valid_err
        counter += 1

# save to csv file

#savetxt('data/weight/weights.csv', weights, delimiter=',')
#savetxt('data/weight/ranks.csv', ranks, delimiter=',')
#savetxt('data/weight/train_err_models.csv', train_err_models, delimiter=',')
#savetxt('data/weight/valid_err_models.csv', valid_err_models, delimiter=',')

## Run optimal models and compare with test set

In [ ]:
'''% TT Reg
% poly order 2
% rank 5
% err 0.3025366067886352539

% TR Reg
% poly order 2
% rank 9
% err 0.3025366067886352539

% CP Reg
% poly order 2
% rank 14
% err 0.3213250637054443359'''

In [ ]:
def regression_test(net, X_test, y_test, n_features):
    
    n_test_samples = len(X_test[:,0])
    X_test = vandermonde_vec(X_test, n_test_samples, n_features, net.poly_order)
    # reshape X_train, X_val to 3d vandermonde for tensor ring and cpd
    if (net.type == 'tr') or (net.type == 'cp'):
        X_test = np.reshape(X_test, (n_test_samples, n_features, net.poly_order))
    
    data = torch.Tensor(X_test)
    targets = torch.Tensor(y_test)
    targets = Variable(targets.long())
    output = net(data, n_test_samples).view(n_test_samples)

    test_err = mean_squared_error(np.asarray(output.data.cpu().numpy()), np.asarray(targets))

    print("\nTest set MSerror:",test_err)
    #for name, param in net.named_parameters():
        #if param.requires_grad:
            #print(name)
            #print(param.data.size())
            #print(param.data)
    return test_err

In [ ]:
# TT
num_epochs = 100
poly_order = 2
rank = 5
num_output = 1

net = TTNet(int(n_features), int(poly_order), int(num_output), int(rank))
optimizer = optim.LBFGS(net.parameters(), line_search_fn='strong_wolfe')
criterion = nn.MSELoss()
#net.to(device)
model_trained, train_err, valid_err, valid_err_cur_best, min_val_output = train_regression_model(
            net, optimizer, criterion, num_epochs, n_features, X_train, X_val, y_train, y_val)

savetxt('data/min_val_output_tt.csv', min_val_output, delimiter=',')

In [ ]:
# TR
num_epochs = 20
poly_order = 2
rank = 9
num_output = 1

net = TRNet(n_features, poly_order, num_output, rank)
optimizer = optim.LBFGS(net.parameters(), line_search_fn='strong_wolfe',max_iter=20)
#optimizer = optim.Adam(net.parameters(), lr=0.1)
criterion = nn.MSELoss()
#net.to(device)
model_trained, train_err, valid_err, valid_err_cur_best, min_val_output = train_regression_model(
            net, optimizer, criterion, num_epochs, n_features, X_train, X_val, y_train, y_val)

savetxt('data/min_val_output_tr.csv', min_val_output, delimiter=',')

In [ ]:
# CP
num_epochs = 100
poly_order = 2
rank = 14
num_output = 1

net = CPNet(int(n_features), int(poly_order), int(num_output), int(rank))
optimizer = optim.LBFGS(net.parameters(), line_search_fn='strong_wolfe')
criterion = nn.MSELoss()
#net.to(device)
model_trained, train_err, valid_err, valid_err_cur_best, min_val_output = train_regression_model(
            net, optimizer, criterion, num_epochs, n_features, X_train, X_val, y_train, y_val)

savetxt('data/min_val_output_cp.csv', min_val_output, delimiter=',')